In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'nithan-chadok-hybrid-ocr-ner:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71848%2F7895361%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240310%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240310T075021Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1487fbd63cc530df920a87112a6bf72c0505f47a3c7d92087251ded30eb64474a33fc190fe698c3380ddbf9f1500d46b172a99798455088d42a39ebfe8badc6b388b4113bbb28d36787df1334fe0d323705bde4a5404f51499eda27eac761c1d9e5c0159e56af58ed0f9cf44e4f6191f201dced2fe9ba2c99735820fa0795ef9b320184e0fb5e72733ed42939573e2d41854e5e7e7ced8790479672d1d59e3a032128985cc046dacc1cdaee8338996251656497b4b014ed69f4e33afd2b739618b1515384bbb294d52319734593904ad362537dc7a187a0870815456e5ad031db7a8115a46988b0049242578001a3b9f72c51c1425b146e4f0529cb8ec974a4f,testdata-teserac:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4564016%2F7795796%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240310%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240310T075021Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db8dd231729668f62bbb523cc0ad92a7b1b0881616737c39bdcdbd3c8ce6d74235716f9a6acc0f20e7538a59461147d9873933c486cb4049ef4b9ef063286f659c71f293253dc727a20c961755e73cc40e41d5c663c58a445533c7a80141c430620dba9eb868abcda15576a0c0f63ec244154d1f2a5f3ce3b7be5a8e365f6e6c391684dd4d0b50d341db76bf2358e281a1f62522ab0d32bb9d751ec10bfd0e38175defc31dc0a5f4af4a86e2f752ba777890994a921db17293eeaf48f8f7905f809af0a76f7673741ec9776127ffe833fc02fb824586777951ff3b74150b2e43aa1315655d8827f2c8f40193e6197ead0e27f754ce137352d517a5f045a9b3618,thapat-210:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4564327%2F7796267%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240310%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240310T075021Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4abcb5de2ca7e3cffd07a196b389fe77dbc0a5ce91adcc4c1593dc2e7be695ccc436d56e6e68f50e47bb3b5d0eb6faad3218e43885bc63134a7a8119f1b91e86cbdbe63dadc123c9b3b727e753eb15214483246f69a95a75d4a258e9421ad52892a342e8e3de839736fab079b55fd849e2526b082d0c4aa8f54b3868f781d6d2b3256e19207fcfa2d6b3cd146b1ad0a6dcd001c15ff173036f734fdcd0ed9a16453642e6ee77fb904f1ce59154709fefc2ffa92793d8955b76403cbd7a0cd8dbf83746bd381a278639699a9439e4efa53eaab5b5382bd07285e74a3236088b7be2a5cad37304acf09f773c4804623b3b79b5155d54e4b02f41e8027a56c984eb,deskew-combine:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4564433%2F7796414%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240310%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240310T075021Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc00c874180bbf2ed041fea4717d94231f617280d9b080275e16a60a9599c6b63710b029d814f2f78b0ab9f94eedf7e7fbdedc6b72c74d8d2c1ccf25427ed4d8897b2fa08494c8c188ead77ff648b6d3d86728e9d466cb12b76acb7a7628432d961e5682ecdddf12a39814c99c348557beaf6300dfc5e9e3ea95d3d7c490d11f040a18525fa192d1ed1b8e6e6e034f4cab61d39e300d3f353d2764a8f1b496bee4a973f313bea56667fb69222d366f2731d8ee69e835e467a9c5c0ae72561c28c88b63069f5d81cb58e117b83b8a796d03e16f8586813108b7c91754808a8ffbeebbd638be8aca635330dc4de27097d40ccc8685fd0fb7f026b5791a87634e1e0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install -q pytesseract

In [ ]:
! sudo apt-get install tesseract-ocr-tha

In [ ]:
! sudo tesseract --list-langs

In [ ]:
import pytesseract
from tqdm import tqdm

# Preprocess

In [ ]:
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, pipeline

tokenizer = CamembertTokenizerFast.from_pretrained("thanaphatt1/WangchanBERTa-LST20")
model = CamembertForTokenClassification.from_pretrained("thanaphatt1/WangchanBERTa-LST20")

nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy="none", ignore_labels=[])

In [ ]:
from PIL import Image
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import os
import pandas as pd
import math

In [ ]:
tag_list = pd.read_csv('/kaggle/input/nithan-chadok-hybrid-ocr-ner/tag_list.csv')
tags = {row['tag']:row['class'] for _, row in tag_list.iterrows()}
src = '/kaggle/input/nithan-chadok-hybrid-ocr-ner/images/images'
kernel = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [0,0,0]
])
preds = []

In [ ]:
from PIL import Image
image = Image.open('/kaggle/input/nithan-chadok-hybrid-ocr-ner/images/images/00001.jpg')
image

In [ ]:
output_tesseract = pytesseract.image_to_string(image, lang='tha')
print(output_tesseract)

In [ ]:
output_tesseract.split('\n')

In [ ]:
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = np.array(cvImage)
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=2)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    firstbox = []
    lastbox = []
    box = []
    for c in contours:
        rect = cv2.boundingRect(c)
        x,y,w,h = rect
        box.append([x,y,x+w,y+h])
        cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

    s_box = sorted(box, key = lambda x: x[1])
    f_box = s_box[0]
    l_box = s_box[-1]
    if -f_box[0]+l_box[0] == 0:
        angle = 0
    else:
        m = (-l_box[1]+f_box[1])/(-f_box[0]+l_box[0])
        degree = math.atan(m)
        angle = degree * 180 / math.pi

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)
#     cv2.imwrite("boxes.jpg", newImage)
    if angle <0:
        angle += 90
    elif angle > 0:
        angle -= 90

    return angle

In [ ]:
getSkewAngle(image)

In [ ]:
plt.imshow(Image.open('/kaggle/working/boxes.jpg'))

In [ ]:
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = np.array(cvImage)
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage
def deskew(cvImage):
    angle = getSkewAngle(cvImage)
    return rotateImage(cvImage, -1.0 * angle)

In [ ]:
plt.imshow(deskew(image))

In [ ]:
# image = Image.open('/kaggle/input/nithan-chadok-hybrid-ocr-ner/images/images/00000.jpg')
inputcopy = deskew(image)
grayinput = cv2.cvtColor(inputcopy, cv2.COLOR_BGR2GRAY)
_, binaryImage = cv2.threshold(grayinput, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
kernelSize = (5, 5)
opIterations = 2
morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
dilateImage = cv2.morphologyEx(
    binaryImage,
    cv2.MORPH_DILATE,
    morphKernel,
    None,
    None,
    opIterations,
    cv2.BORDER_REFLECT101
)

contours, _ = cv2.findContours(dilateImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mean_row = np.array([c[:, 0].mean(0)[1] for c in contours])
min_row = np.array([c[:, 0].min(0)[1] for c in contours], dtype='int')
max_row = np.array([c[:, 0].max(0)[1] for c in contours], dtype='int')
min_col = np.array([c[:, 0].min(0)[0] for c in contours], dtype='int')
max_col = np.array([c[:, 0].max(0)[0] for c in contours], dtype='int')
row_idx = np.argsort(mean_row)
words = []

for idx in range(len(row_idx)):
    xx = cv2.drawContours(np.zeros_like(image, dtype='uint8'), contours, row_idx[idx], (1, 1, 1), thickness=-1)
    im = (1 - inputcopy) * xx
    im = im[min_row[row_idx[idx]]:max_row[row_idx[idx]], min_col[row_idx[idx]]:max_col[row_idx[idx]]]
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    sharpened = cv2.filter2D(1 - gray, -1, kernel)

    thresh1 = (gray < 100).astype('uint8') * 255
    thresh2 = cv2.adaptiveThreshold(sharpened, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)
    padded = cv2.copyMakeBorder(thresh1.copy(), 10, 10, 10, 10, cv2.BORDER_CONSTANT,value=(255,))
    word = pytesseract.image_to_string(padded, lang='tha').replace("_","[!und:]").replace(" ","[!und:]").replace('\x0c', '').replace('\n', '')
    print(word)
    if len(word) == 0:
        word = '[!und:]'
    words.append(word)
    preds.append(nlp(word)[0]['entity'])
words, preds

## read each word

In [ ]:
import pytesseract

def crop_image(image):
    grayinput = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binaryImage = cv2.threshold(grayinput, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernelSize = (5, 5)
    opIterations = 2
    morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)
    dilateImage = cv2.morphologyEx(
        binaryImage,
        cv2.MORPH_DILATE,
        morphKernel,
        None,
        None,
        opIterations,
        cv2.BORDER_REFLECT101
        )
    contours, _ = cv2.findContours(dilateImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mean_row = np.array([c[:, 0].mean(0)[1] for c in contours])
    min_row = np.array([c[:, 0].min(0)[1] for c in contours], dtype='int')
    max_row = np.array([c[:, 0].max(0)[1] for c in contours], dtype='int')
    min_col = np.array([c[:, 0].min(0)[0] for c in contours], dtype='int')
    max_col = np.array([c[:, 0].max(0)[0] for c in contours], dtype='int')
    row_idx = np.argsort(mean_row)
    return row_idx, contours, min_row, max_row, min_col, max_col

def image_to_words(row_idx, contours, min_row, max_row, min_col, max_col, image):
    words = []
    for idx in range(len(row_idx)):
        im = image[min_row[row_idx[idx]]:max_row[row_idx[idx]], min_col[row_idx[idx]]:max_col[row_idx[idx]]]
        word = pytesseract.image_to_string(im, lang='tha').replace("_","[!und:]").replace(" ","[!und:]").replace('\x0c', '').replace('\n', '')
        if len(word) == 0:
            word = '-'
        words.append(word)
    return words

In [ ]:
image = Image.open("/kaggle/input/nithan-chadok-hybrid-ocr-ner/images/images/00001.jpg")
image = np.array(image)
inputcopy = deskew(image)
row_idx, contours, min_row, max_row, min_col, max_col = crop_image(inputcopy)

num_rows = 2
num_cols = 7
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 3))

for idx, ax in enumerate(axes.flat):
    if idx < len(row_idx):
        im = inputcopy[min_row[row_idx[idx]]:max_row[row_idx[idx]], min_col[row_idx[idx]]:max_col[row_idx[idx]]]
        ax.imshow(im, cmap='gray')
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
words = image_to_words(row_idx, contours, min_row, max_row, min_col, max_col, image)
print(words)

# submit

## transform & load data

In [ ]:
src = '/kaggle/input/nithan-chadok-hybrid-ocr-ner/images/images'
txt = []

for i in tqdm(range(len(os.listdir(src)))):
    image = Image.open(f'{src}/{i:05d}.jpg')
    image = np.array(image)
    inputcopy = deskew(image)
    row_idx, contours, min_row, max_row, min_col, max_col = crop_image(inputcopy)
    words = image_to_words(row_idx, contours, min_row, max_row, min_col, max_col, inputcopy)
    txt.extend(words)

print(len(txt))

In [ ]:
df = pd.DataFrame({'Text': txt})
df.to_csv('text_data.csv', index=False)

## read text data

In [ ]:
NER_TAGS = [
       "O",
        "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "B_ORG",        "B_PER",        "B_TRM",        "B_TTL",
       "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
        "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "E_ORG",        "E_PER",        "E_TRM",        "E_TTL"]

In [ ]:
# txt = pd.read_csv('/kaggle/input/testdata-teserac/text_data.csv')
txt = pd.read_csv('/kaggle/input/deskew-combine/text_combine_norm1_de2.csv')
txt = txt['txt']
txt

## read model

In [ ]:
!pip install -q simpletransformers

In [ ]:
import torch
from simpletransformers.ner import NERModel, NERArgs

ner_args = NERArgs()
ner_args.eval_batch_size = 8
ner_args.use_multiprocessing = True
ner_args.max_seq_length = 512
model = NERModel(
     "auto", "thanaphatt1/WangchanBERTa-LST20", args=ner_args, use_cuda=torch.cuda.is_available(), labels= NER_TAGS
)

In [ ]:
def split_into_sentences(tokens, tokens_per_sentence):
    sentences = []
    for i in range(0, len(tokens), tokens_per_sentence):
        sentence = tokens[i:i+tokens_per_sentence]
        sentences.append(sentence)
    return sentences

## token & prediction

In [ ]:
test_tokens_1 = split_into_sentences(txt, 224)
pred_1 = model.predict(test_tokens_1, False)

In [ ]:
test_tokens_2 = split_into_sentences(txt, 196)
pred_2 = model.predict(test_tokens_2, False)

In [ ]:
test_tokens_3 = split_into_sentences(txt, 210)
pred_3 = model.predict(test_tokens_3, False)

In [ ]:
tag_df = pd.read_csv('/kaggle/input/nithan-chadok-hybrid-ocr-ner/tag_list.csv')
tag_df

### compare

In [ ]:
final_1 = []
final_2 = []
final_3 = []

for i in pred_1[0]:
    for j in i:
        for k in j.values():
            result = tag_df[tag_df['tag'] == k]['class'].values[0]
            final_1.append(result)

for i in pred_2[0]:
    for j in i:
        for k in j.values():
            result = tag_df[tag_df['tag'] == k]['class'].values[0]
            final_2.append(result)

for i in pred_3[0]:
    for j in i:
        for k in j.values():
            result = tag_df[tag_df['tag'] == k]['class'].values[0]
            final_3.append(result)

print(len(final_1))
print(len(final_2))
print(len(final_3))

In [ ]:
submit_1 = pd.read_csv('/kaggle/input/nithan-chadok-hybrid-ocr-ner/sample_submission.csv')
submit_2, submit_3 = submit_1.copy(), submit_1.copy()
# submit_3 = pd.read_csv('/kaggle/input/thapat-210/thapat_210.csv')

submit_1

In [ ]:
submit_1['pred'] = pd.DataFrame(final_1)
submit_2['pred'] = pd.DataFrame(final_2)
submit_3['pred'] = pd.DataFrame(final_3)
submit_1

In [ ]:
compare = pd.concat([submit_1['Id'], submit_1['pred'], submit_2['pred'], submit_3['pred']], axis=1)
compare.columns=['id','224','196', '210']

In [ ]:
compare[compare['224'] != compare['196']]

In [ ]:
compare[compare['224'] != compare['210']]

In [ ]:
compare[compare['196'] != compare['210']]

In [ ]:
submit_1.to_csv('deskew_224.csv', index=False)

In [ ]:
submit_2.to_csv('thapat_196.csv', index=False)

In [ ]:
submit_3.to_csv('thapat_210.csv', index=False)

In [ ]:
import torch
torch.cuda.empty_cache()
!rm -r /kaggle/working/*

### ensemble mod

In [ ]:
df_ensemble = pd.concat([submit_1, submit_2, submit_3])
df_ensemble = df_ensemble.groupby("Id")["pred"].apply(lambda x: x.mode()[0]).reset_index()
df_ensemble

In [ ]:
compare_2 = pd.concat([df_ensemble['Id'], df_ensemble['pred'], submit_1['pred']], axis=1)
compare_2.columns=['id','ens', '224']

In [ ]:
compare_2[compare_2['ens'] != compare_2['224']]

In [ ]:
df_ensemble.to_csv('deskew_224_196_210.csv', index=False)